In [1]:
from azureml.core import Workspace
from obs.management import provision,set_adx_to_workspace
import pandas as pd
from datetime import timedelta
from obs.collector import Online_Collector

ws = Workspace.from_config()

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyarrow 7.0.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('pyarrow<4.0.0,>=0.17.0'), {'azureml-dataset-runtime'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core

### Provision Resources

#### Create ADX Cluster, Service Principal, and store value's in Azure Machine Learning's Key Vault for future use

In [ ]:
#Provision with default SKU of Standard_D11_v2, Standard Tier of ADX cluster. The provion will also create a service principal as part of the process.
#You need to have right to provision service principal in the subscription.
provision(ws)

#### Setup a service principal, ADX cluster, database in seperate step and attach the ADX cluster to Azure ML

1. Setup ADX cluster:
- Create your own ADX cluster. The cluster has to be enabled with Streaming Ingestion (https://docs.microsoft.com/en-us/azure/data-explorer/ingest-data-streaming?tabs=azure-portal%2Ccsharp) and Python language extension https://docs.microsoft.com/en-us/azure/data-explorer/language-extensions
- Create or reuse an existing service principal
- Create a database
- Assign the service principal to the database as admin https://docs.microsoft.com/en-us/azure/data-explorer/manage-database-permissions


2. Attach the cluster to Azure ML workspace
Prepare cluster_uri (e.g. https://adx02.westus2.kusto.windows.net),db_name, client_id, client_secret, subscription_id, tenant_id and run the following command to attach the cluster to Azure ML workspace.

In [ ]:
set_adx_to_workspace(ws, cluster_uri,db_name, client_id, client_secret, subscription_id, tenant_id)


### Data Collection

Once the resources are created, data can now be ingested to Azure Data Explorer. To use the dashboards, the data must have a timestamp column. 

In [2]:
# Get raw data

dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")

In [5]:
# Add timestamp column
from obs.collector import Online_Collector
import pandas as pd
dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")

dataset["timestamp"] =  [pd.to_datetime('now') - timedelta(days=x) for x in range(len(dataset))]
table_name = "IRIS_DATA_NEW" #new dataset

online_collector = Online_Collector(table_name,ws=ws)
online_collector.batch_collect(dataset)

.create table IRIS_DATA_NEW (['sepal_length']: real, ['sepal_width']: real, ['petal_length']: real, ['petal_width']: real, ['species']: string, ['timestamp']: datetime)


#### Batch Ingestion

Entire dataframe will be loaded into ADX at once as a table named ```irisdata```. The is also a stream ingestion available to ingest data asynchronously with an internal buffering mechanism. This method can be utilized to lower impact to main scoring thread.

In [4]:

table_name = "irisdata" #new dataset

online_collector = Online_Collector(table_name,ws=ws)
online_collector.batch_collect(dataset)

.create table irisdata (['sepal_length']: real, ['sepal_width']: real, ['petal_length']: real, ['petal_width']: real, ['species']: string, ['datetime']: datetime)


### Spark Ingestion (run this in Databricks or Synapse)

In Databricks or Synapse Spark, install the library:

```pip install --upgrade git+https://github.com/microsoft/AzureML-Observability#subdirectory=aml-obs-collector```



#### Logon with a service principal so that you can run this as a job. You can also logon w9th interactive  mode

In [ ]:

from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core import Workspace

sp_auth = ServicePrincipalAuthentication(tenant_id =tenant_id,
                                       service_principal_id=service_principal_id,
                                       service_principal_password=service_principal_password)
# Instantiate Azure Machine Learning workspace
ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,auth= sp_auth)

#### Load a Spark Dataframe

In [ ]:
data =spark.read.format("csv").option("header", True).load("wasbs://ojsales-simulatedcontainer@azureopendatastorage.blob.core.windows.net/oj_sales_data/Store10*.csv")

#### Ingest!

In [ ]:
from obs.collector import spark_collect

table_name = "adb_oj_sales"
spark_collect(data,table_name,ws)
#will take a few minutes for result to show up in ADX

## Real time ingestion

Check out the example in monitoring notebook to see how real time ingestion works